# Домашняя работа

### 1. Доделать все функции, где стоит комментарий "сделать дома"

**Hitrate**

In [1]:
import pandas as pd
import numpy as np

In [4]:
def hit_rate(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    hit_rate = (flags.sum() > 0) * 1
    
    return hit_rate


def hit_rate_at_k(recommended_list, bought_list, k=5):
    return hit_rate(recommended_list=recommended_list[:k], 
                   bought_list=bought_list)

In [8]:
recommended_list = [143, 523, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [143, 523, 1134, 991]

hit_rate(recommended_list, bought_list), hit_rate_at_k(recommended_list, bought_list)

(1, 1)

**Precision**

In [11]:
from IPython.core.debugger import set_trace

prices_recommended = [round(np.random.uniform(1, 100), 3) for _ in recommended_list]
prices_recommended

[32.646,
 69.987,
 17.996,
 15.301,
 56.344,
 27.649,
 18.483,
 32.358,
 92.605,
 10.769]

In [14]:
def precision(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision


def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision


def money_precision_at_k(recommended_list, bought_list, prices_recommended, k=5):
        
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    prices_recommended = np.array(prices_recommended[:k])
    
    flags = np.isin(recommended_list, bought_list)
    
    precision = np.dot(flags, prices_recommended) / np.sum(prices_recommended)
    
    return precision

money_precision_at_k(recommended_list, bought_list, prices_recommended)

0.7069598593673612

**Recall**

In [17]:
bought_list = [143, 523, 1134, 991, 1, 2, 3]
prices_bought = [32.646, 69.987, 17.996, 15.301, 30, 50, 100]

In [19]:
def recall(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall


def recall_at_k(recommended_list, bought_list, k=5):
    return recall(recommended_list[:k], bought_list)

# (revenue of recommended items @k that are relevant) / (revenue of relevant items)
def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    prices_recommended = np.array(prices_recommended[:k])
    prices_bought = np.array(prices_bought)
    
    flags = np.isin(recommended_list, bought_list)
    
    recall = np.dot(prices_recommended, flags) / np.sum(prices_bought)
    
    return recall

money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought)

0.4302535371759567

**ap@k**

In [22]:
def ap_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    relevant_indexes = np.nonzero(np.isin(recommended_list, bought_list))[0]
    
    if len(relevant_indexes) == 0:
        return 0
    
    amount_relevant = len(relevant_indexes)
    
    sum_ = sum([precision_at_k(recommended_list, bought_list, k=index_relevant+1) for index_relevant in relevant_indexes])
    return sum_ / amount_relevant

**map@k**

In [32]:
# теперь список из 3 пользователей
recommended_list_3_users = [[143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43], 
                    [1134, 533, 14, 4, 15, 1543, 1, 99, 27, 3345],
                    [991, 3345, 27, 533, 43, 143, 1543, 156, 1134, 11]
                           ]

bought_list_3_users = [[521, 32, 143],  # юзер 1
                       [143, 533, 4, 43, 11], # юзер 2
                       [991, 2]] # юзер 3



In [37]:
def map_k(recommended_list, bought_list, k=5):
#     set_trace()
    result = np.array(
        list(map(lambda x: ap_k(x[0], x[1], k=k), zip(recommended_list_3_users, bought_list_3_users)))
    )
    
    return result.mean()

map_k(recommended_list_3_users, bought_list_3_users)

0.8333333333333334

**ndcg@k**

In [38]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43]
bought_list = [521, 32, 143]

In [41]:
def ndcg_at_k(recommended_list, bought_list, k=5):
    def dcg(flags):
        result = np.array([
            [flag / i if i <=2 else flag / np.log2(i) for i, flag in enumerate(flags, start=1)]
        ])
        return np.mean(result)
    
    recommended_list = np.array(recommended_list[:k])
    bought_list = np.array(bought_list)
    
    flags = np.isin(recommended_list, bought_list)
    
    return dcg(flags) / dcg([1 for _ in flags])


ndcg_at_k(recommended_list, bought_list)
   

0.3266259271143027

**MRR@K**

In [61]:
# теперь список из 3 пользователей
recommended_list_3_users = [[143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43], 
                    [1134, 533, 14, 4, 15, 1543, 1, 99, 27, 3345],
                    [991, 3345, 27, 533, 43, 143, 1543, 156, 1134, 11]]

bought_list_3_users = [[143, 32, 143],  # юзер 1
                       [23, 156, 991, 14, 11], # юзер 2
                       [991,2]] # юзер 3

In [62]:
def mean_reciprocal_rank(recommended_list, bought_list, k=3):
    def reciprocal_rank(recommended_list, bought_list):
        recommended_list = np.array(recommended_list[:k])
        bought_list = np.array(bought_list)
        
        first_isin = np.nonzero(np.isin(recommended_list, bought_list))[0]
        first_isin = np.where(first_isin == 0, first_isin + 1, first_isin)
        
        return 0 if not first_isin else 1 / first_isin
    
    return np.mean(list(map(
        lambda x: reciprocal_rank(x[0], x[1]),
        zip(recommended_list, bought_list)
    )))

mean_reciprocal_rank(recommended_list_3_users, bought_list_3_users)

0.8333333333333334

### 2. Приведите еще примеры метрик для оценки рекомендаций/ранжирования (можно взять из интернета, или ваши знания)

**CG@K (Cumulative Gain at K)**

$$CG@K=\sum_{k=1}^K{r^{\text{true}}(\pi^{-1}(k))}$$

*Плюсы:*
* подходит и для не бинарной классификации

*Минусы*
* не нормализована
* не учитывает позицию релевантных векторов

Пример

In [63]:
r_true = [1, 0, 1, 0] #флаги покупок
cg_at_k = sum(r_true)
cg_at_k

2

**Discounted Cumulative Gain at K**

$$DCG@K=\sum_{k=1}^K\frac{2^{r^{\text{true}}(\pi^{-1}(k))}-1}{\log_2(k+1)}$$

*Плюсы:*
* подходит и для не бинарной классификации
* ранжированная метрика

*Минусы*
* не нормализована

**Expected reciprocal rank (ERR)**

$$ERR@K=\sum_{k=1}^{K}\frac{1}{k}P(\text{обьект остановиться на элементе с рангом k})$$

$$P(\text{обьект остановиться на элементе с рангом k})=p_k\prod_{i=1}^{k-1}(1-p_i)$$

где $p_k$ — вероятность того, что пользователь будет удовлетворен объектом с рангом $r^{\text{true}}$. 

$$p_k=\frac{2^{r^{\text{true}}(\pi^{-1}(k))}-1}{2^{\text{max}r^{\text{true}}(\pi^{-1}(k))}-1}$$

**PFound**

$$PFound@K=\sum_{k=1}^kPLook(k)\cdot PRel(k)$$

$$PLook(k)=PLook(k-1)(1-PRel(k-1))(1-PBreak)$$
$$PRel=2^{r^{\text{true}}(\pi^{-1}(k))}-1$$
$$PBreak=\text{вероятность того, что пользователь прекратит просмотр по внешним причинам}$$